## Azure AI Red Teaming (Slim Notebook)

Objective: Run a minimal, auditable red teaming workflow against a single Azure AI (AIServices) Project using only Managed Identity (no API keys, no legacy hub/workspace fallback) to generate safety signals (Attack Success Rate) across selected risk categories and attack strategies.

Prerequisites (outside this notebook):
- Azure AI Foundry Project connected to a storage account (Entra ID auth; project identity has at least Storage Blob Data Contributor)
- Project identity has required evaluation roles (e.g., AzureML Data Scientist) so evaluations & artifacts log successfully
- Azure OpenAI deployment (or adjust callbacks to your own target service)
- Notebook running on Azure ML / compute with the same Managed Identity enabled

Design Principles:
- Fail fast on misconfiguration (no silent credential or legacy fallbacks)
- Keep surface minimal (≤ ~30 cells) with clear section intent
- Emphasize readability & extension points over exhaustive diagnostics (deep forensic cells moved out)

Section Map:
1. Bootstrap & Config
2. Project Reachability Probe (non-blocking API visibility check)
3. Minimal Single-Category Scan (sanity + discovery of portal links)
4. Basic Callback Scan (simple deterministic app callback)
5. Intermediary Managed Identity Model Callback Scan
6. Advanced Multi‑Strategy Model Scan
7. Custom Prompts (bring-your-own objectives)
8. Readiness / Health Check (rerunnable)
9. Conclusion & Next Steps

If you need full diagnostics (token scopes, MLflow/artifact instrumentation, RBAC enumeration), refer to the archived reference notebook.


In [24]:
# Bootstrap: install core packages (idempotent) and load config
import os, sys, subprocess, json, datetime as dt, asyncio, pathlib

REQUIRED = [
    'azure-ai-evaluation[redteam]',
    'azure-ai-projects',
    'azure-identity',
    'openai'
]
for pkg in REQUIRED:
    try:
        __import__(pkg.split('[')[0].replace('-', '_'))
    except Exception:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', '--quiet', pkg])

from azure.identity import ManagedIdentityCredential, get_bearer_token_provider
from azure.ai.evaluation.red_team import RedTeam, RiskCategory, AttackStrategy
from openai import AzureOpenAI

# --- Hardcoded Environment (dev) ---
# Pulled from live Azure inventory via MCP queries (2025-08-10). Adjust only when promoting to another environment.
SUBSCRIPTION_ID = 'e440a65b-7418-4865-9821-88e411ffdd5b'
RESOURCE_GROUP = 'rg-thorlabs-redteam-dev-eastus2'
# AIServices Project target
PROJECT_NAME = 'thorlabs-aisvcproj-redteam-dev'
# AIServices Account + subdomain
AISVCS_ACCOUNT = 'thorlabs-aisvc-redteam-dev'
AISVCS_SUBDOMAIN = 'thorlabsredteamdev'
# Azure OpenAI (kind=OpenAI) endpoint + deployment
AZURE_OPENAI_ENDPOINT = 'https://thorlabs-openai-redteam-dev.openai.azure.com/'
AZURE_OPENAI_DEPLOYMENT = 'gpt-4o-mini-redteam'
AZURE_OPENAI_API_VERSION = '2024-08-01-preview'
# Legacy ML workspace project (fallback control target) - retained for comparative control runs
LEGACY_PROJECT_NAME = 'thorlabs-project-redteam-dev'

# Managed Identity preferences
UAMI_CLIENT_ID = '95aedfd4-301c-4105-a6db-0a83c9fd5ddd'  # user-assigned MI (stable)
FORCE_SYSTEM_MI = os.environ.get('FORCE_SYSTEM_MI', '0') == '1'
DIAG = os.environ.get('DIAG', '0') == '1'

missing = [k for k,v in dict(PROJECT_NAME=PROJECT_NAME, AZURE_OPENAI_ENDPOINT=AZURE_OPENAI_ENDPOINT, AZURE_OPENAI_DEPLOYMENT=AZURE_OPENAI_DEPLOYMENT).items() if not v]
if missing:
    raise ValueError(f"Missing required config vars: {missing}")

# Enforce managed identity only; prefer UAMI unless explicitly overridden
try:
    if FORCE_SYSTEM_MI:
        credential = ManagedIdentityCredential()
        active_identity = 'system-assigned'
    else:
        credential = ManagedIdentityCredential(client_id=UAMI_CLIENT_ID)
        active_identity = f'uami:{UAMI_CLIENT_ID}'
    _tok = credential.get_token('https://cognitiveservices.azure.com/.default')
except Exception as e:
    raise RuntimeError(f"ManagedIdentityCredential unavailable: {e}. Attach to Azure ML compute with MI enabled.")

azure_ai_project = {
    'subscription_id': SUBSCRIPTION_ID,
    'resource_group_name': RESOURCE_GROUP,
    'project_name': PROJECT_NAME,
}
print(f"[config] Project={PROJECT_NAME} Deployment={AZURE_OPENAI_DEPLOYMENT} identity={active_identity}")

# Token provider + client for model callbacks
_token_provider = get_bearer_token_provider(credential, 'https://cognitiveservices.azure.com/.default')
client = AzureOpenAI(
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_version=AZURE_OPENAI_API_VERSION,
    azure_ad_token_provider=_token_provider,
)

def model_callback(messages, stream: bool|None=False, **_):  # sync wrapper; RedTeam supports callable
    latest = messages[-1].content if hasattr(messages[-1], 'content') else getattr(messages[-1], 'text', '')
    try:
        resp = client.chat.completions.create(
            model=AZURE_OPENAI_DEPLOYMENT,
            messages=[{'role':'user','content': latest}],
            max_completion_tokens=200,
            temperature=0.5,
        )
        return {'messages':[{'role':'assistant','content': resp.choices[0].message.content}]}
    except Exception as e:
        return {'messages':[{'role':'assistant','content': f'[error] {e}'}]}

# Simple fixed safe callback for baseline demonstration
def fixed_safe_callback(_prompt: str|None=None, **_):
    return "This system cannot assist with unsafe requests and will provide only safe guidance."  # deterministic

# Prepare local output directory
output_dir = pathlib.Path('/tmp/ai-redteam-logs')
output_dir.mkdir(parents=True, exist_ok=True)
print(f"[bootstrap] Ready. output_dir={output_dir} legacy_project={LEGACY_PROJECT_NAME}")

[config] Project=thorlabs-aisvcproj-redteam-dev Deployment=gpt-4o-mini-redteam identity=uami:95aedfd4-301c-4105-a6db-0a83c9fd5ddd
[bootstrap] Ready. output_dir=/tmp/ai-redteam-logs legacy_project=thorlabs-project-redteam-dev


In [25]:
# Project reachability check (fail fast – no legacy fallback)
import requests, json
ACCOUNT_SUBDOMAIN = os.environ.get('AISVCS_SUBDOMAIN', 'thorlabsredteamdev')
base = f"https://{ACCOUNT_SUBDOMAIN}.services.ai.azure.com"
project_url = f"{base}/api/projects/{PROJECT_NAME}"
list_url = f"{base}/api/projects"

# Attempt data-plane discovery (expected may 404 if API surface not yet exposed; surface clearly)
scopes = ['https://cognitiveservices.azure.com/.default']
try:
    token = credential.get_token(scopes[0]).token
    hdr = {'Authorization': f'Bearer {token}'}
    r_list = requests.get(list_url, headers=hdr, timeout=15)
    r_proj = requests.get(project_url, headers=hdr, timeout=15)
    print(f"[reachability] LIST {r_list.status_code} PROJ {r_proj.status_code}")
    if r_proj.status_code == 404:
        print('[reachability] Project endpoint 404. Continuing because current SDK may not rely on this path; no silent substitution performed.')
    elif r_proj.ok:
        try:
            print('[reachability] Project payload keys:', list(r_proj.json().keys())[:10])
        except Exception:
            pass
except Exception as e:
    print('[reachability] token/HTTP failure:', e)

print('Proceed to Basic Scan section.')

[reachability] LIST 404 PROJ 404
[reachability] Project endpoint 404. Continuing because current SDK may not rely on this path; no silent substitution performed.
Proceed to Basic Scan section.


In [ ]:
# Token & Scope Diagnostics (401 Triage)
# Purpose: Differentiate credential issues vs scope mismatch vs RBAC after restart.
# Run after the Bootstrap & Reachability cells if you see 401 or 403 constructing RedTeam or hitting project endpoints.
import time, os, json, base64, textwrap, requests
from azure.identity import ManagedIdentityCredential

print('[diag] Active identity variable:', globals().get('active_identity'))
force_system = globals().get('FORCE_SYSTEM_MI', False)
client_id = globals().get('UAMI_CLIENT_ID')

# Build credential exactly like bootstrap
try:
    if force_system:
        cred_test = ManagedIdentityCredential()
    else:
        cred_test = ManagedIdentityCredential(client_id=client_id)
except Exception as e:
    raise RuntimeError(f"ManagedIdentityCredential init failure: {e}")

scopes_to_try = [
    'https://cognitiveservices.azure.com/.default',
    'https://ai.azure.com/.default',  # Emerging scope (try if data-plane evolves)
]

ACCOUNT_SUBDOMAIN = os.environ.get('AISVCS_SUBDOMAIN', globals().get('AISVCS_SUBDOMAIN', 'thorlabsredteamdev'))
PROJECT_NAME = globals().get('PROJECT_NAME')
base_ai = f"https://{ACCOUNT_SUBDOMAIN}.services.ai.azure.com"
project_url = f"{base_ai}/api/projects/{PROJECT_NAME}"
list_url = f"{base_ai}/api/projects"

results = []
for scope in scopes_to_try:
    try:
        t0 = time.time()
        tok = cred_test.get_token(scope)
        latency = (time.time() - t0) * 1000
        token_head = tok.token.split('.')[0]
        try:
            padded = token_head + '=' * (-len(token_head) % 4)
            header_dec = base64.urlsafe_b64decode(padded.encode()).decode(errors='ignore')
        except Exception:
            header_dec = '<decode-failed>'
        hdr = {'Authorization': f'Bearer {tok.token}'}
        r_list = requests.get(list_url, headers=hdr, timeout=15)
        r_proj = requests.get(project_url, headers=hdr, timeout=15)
        results.append({
            'scope': scope,
            'expires_in_s': int(tok.expires_on - time.time()),
            'acquire_ms': int(latency),
            'list_status': r_list.status_code,
            'proj_status': r_proj.status_code,
        })
        print(f"[diag] scope={scope} acquire={int(latency)}ms exp_in={int(tok.expires_on - time.time())}s LIST={r_list.status_code} PROJ={r_proj.status_code}")
        if r_proj.status_code in (401, 403):
            print('  body snippet:', textwrap.shorten(r_proj.text, 160))
    except Exception as e:
        print(f"[diag] scope={scope} failed: {e}")

# ARM management-plane check (should remain 200/403, not 401)
try:
    arm_tok = cred_test.get_token('https://management.azure.com/.default')
    arm_hdr = {'Authorization': f'Bearer {arm_tok.token}'}
    acct = globals().get('AISVCS_ACCOUNT')
    SUBSCRIPTION_ID = globals().get('SUBSCRIPTION_ID')
    RESOURCE_GROUP = globals().get('RESOURCE_GROUP')
    proj_id = f"/subscriptions/{SUBSCRIPTION_ID}/resourceGroups/{RESOURCE_GROUP}/providers/Microsoft.CognitiveServices/accounts/{acct}/projects/{PROJECT_NAME}"
    r_arm = requests.get(f"https://management.azure.com{proj_id}?api-version=2025-06-01", headers=arm_hdr, timeout=20)
    print(f"[diag] ARM project GET status={r_arm.status_code}")
    if r_arm.status_code == 401:
        print('[diag] Unexpected 401 on ARM plane -> indicates identity token acquisition issue, not RBAC.')
except Exception as e:
    print('[diag] ARM probe error:', e)

# Classification heuristics
if any(r['proj_status'] == 401 for r in results):
    if all(r['list_status'] == 401 for r in results):
        print('[diag-class] Consistent 401: likely token audience mismatch or identity not fully propagated after restart. Wait 2-3 min and re-run.')
    else:
        print('[diag-class] Mixed statuses: partial scope acceptance; check which scope returns non-401 and adopt that in callbacks.')
elif any(r['proj_status'] == 403 for r in results):
    print('[diag-class] 403 present (RBAC) — role propagation may still be pending. Re-test in a few minutes.')
else:
    print('[diag-class] No 401/403 responses -> if constructor still fails, root cause may be SDK version or unsupported project path.')

print('[diag] Done. Share this output if issues persist.')

### Quick Runbook (Initial Execution)

Use this streamlined path to actually perform the first Azure AI Red Teaming evaluation with the current static configuration (single risk category, single strategy) and confirm artifact logging.

Steps:
1. Run the Bootstrap & Config cell.
2. Run the Project Reachability Probe cell (optional but helps visibility).
3. Run the Quick Runbook Execution cell immediately below.
4. Open any printed scorecard/evaluation URL in the AI Foundry portal.

What it does:
- Performs a management-plane confirmation (optional silent failure tolerated).
- Instantiates a RedTeam object (Violence category, 1 objective) using Managed Identity only.
- Executes a minimal scan against the deterministic safe callback (expected ASR ~0%).
- Prints key identifiers (evaluation_id, scorecard_url) and a local artifact subset.

If construction fails:
- The cell classifies the failure (RBAC vs Unsupported vs Generic) to speed triage.

Once this succeeds, you can proceed to the later (basic/intermediary/advanced) sections. Set the env var DIAG=1 before starting the kernel if you want the deeper diagnostic cells to matter; otherwise, you can ignore them for the quick path.


In [26]:
# Lightweight az CLI bypass: direct Managed Identity REST probes (no az commands)
import json, requests, os
from azure.identity import ManagedIdentityCredential
cred = ManagedIdentityCredential()
sub = SUBSCRIPTION_ID
rg = RESOURCE_GROUP
proj = PROJECT_NAME
account = os.environ.get('AISVCS_ACCOUNT','thorlabs-aisvc-redteam-dev')
# Acquire ARM token for management-plane validation
arm_token = cred.get_token('https://management.azure.com/.default').token
h_arm = {'Authorization': f'Bearer {arm_token}'}
project_arm_id = f"/subscriptions/{sub}/resourceGroups/{rg}/providers/Microsoft.CognitiveServices/accounts/{account}/projects/{proj}"
project_get = f"https://management.azure.com{project_arm_id}?api-version=2023-05-01"  # preview API version for AIServices projects (adjust if GA)
resp = requests.get(project_get, headers=h_arm, timeout=20)
print('[arm project GET]', resp.status_code)
if resp.ok:
    try:
        data = resp.json()
        print('name:', data.get('name'), 'location:', data.get('location'), 'provisioningState:', data.get('properties',{}).get('provisioningState'))
    except Exception:
        print('Non-JSON response snippet:', resp.text[:120])
else:
    print('Body snippet:', resp.text[:160])

# Data-plane probe with MSI (cognitiveservices scope) — already done above; this shows management vs data-plane split.
print('Bypassed az CLI: used pure REST + ManagedIdentityCredential.')

[arm project GET] 400
Body snippet: {"error":{"code":"NoRegisteredProviderFound","message":"No registered resource provider found for location 'eastus2' and API version '2023-05-01' for type 'acco
Bypassed az CLI: used pure REST + ManagedIdentityCredential.


### Model Target Adapter & Legacy Workspace Control Run

Purpose: Provide a real model target (Azure OpenAI or custom HTTPS) and a fallback control evaluation using the legacy ML workspace project to validate SDK/evaluation path independent of AIServices project RBAC.

Set the following environment variables before running the adapter code cell below:
- AZURE_OPENAI_ENDPOINT (already present)
- AZURE_OPENAI_DEPLOYMENT (deployment name, e.g. gpt-4o-mini)
- AZURE_OPENAI_API_VERSION (optional override)
- LEGACY_PROJECT_NAME (ML workspace project name if distinct)
- TARGET_INFERENCE_URL (optional custom HTTPS endpoint instead of Azure OpenAI)

Run order:
1. Bootstrap cell (already executed earlier).
2. Adapter code cell (next) – confirms model reachability.
3. Legacy control run cell – constructs RedTeam against legacy project name and executes minimal scan.
4. Compare artifact/portal success vs AIServices attempt to isolate RBAC vs feature support issues.


In [27]:
# Model adapter: Azure OpenAI (hardcoded) or custom HTTPS endpoint via Managed Identity
import os, json, httpx, pathlib, time
from typing import List, Dict, Any
from azure.identity import ManagedIdentityCredential, get_bearer_token_provider
from openai import AzureOpenAI

# Hardcoded values (dev) aligned with bootstrap
AOAI_ENDPOINT = 'https://thorlabs-openai-redteam-dev.openai.azure.com/'
AOAI_DEPLOYMENT = 'gpt-4o-mini-redteam'
AOAI_API_VERSION = '2024-08-01-preview'
CUSTOM_ENDPOINT = os.getenv('TARGET_INFERENCE_URL')  # remains optional override
LEGACY_PROJECT_NAME = 'thorlabs-project-redteam-dev'

if not (CUSTOM_ENDPOINT or (AOAI_ENDPOINT and AOAI_DEPLOYMENT)):
    raise ValueError("Adapter configuration incomplete.")

credential_adapter = ManagedIdentityCredential(client_id='95aedfd4-301c-4105-a6db-0a83c9fd5ddd')  # prefer UAMI

aoai_client = None
if not CUSTOM_ENDPOINT:
    token_provider = get_bearer_token_provider(credential_adapter, 'https://cognitiveservices.azure.com/.default')
    aoai_client = AzureOpenAI(
        azure_endpoint=AOAI_ENDPOINT,
        api_version=AOAI_API_VERSION,
        azure_ad_token_provider=token_provider,
    )

def invoke_model(messages: List[Dict[str, str]]) -> str:
    if CUSTOM_ENDPOINT:
        token = credential_adapter.get_token('https://management.azure.com/.default').token
        headers = {'Authorization': f'Bearer {token}', 'Content-Type': 'application/json'}
        payload = {'messages': messages}
        with httpx.Client(timeout=60) as client:
            r = client.post(CUSTOM_ENDPOINT, headers=headers, json=payload)
            r.raise_for_status()
            data = r.json()
        return (
            data.get('output')
            or (data.get('choices', [{}])[0].get('message', {}).get('content'))
            or data.get('response')
            or json.dumps(data)[:400]
        )
    else:
        resp = aoai_client.chat.completions.create(
            model=AOAI_DEPLOYMENT,
            messages=messages,
            temperature=0.2,
            max_completion_tokens=256,
        )
        return resp.choices[0].message.content

# Smoke test
try:
    smoke = invoke_model([{'role': 'user', 'content': "Reply with 'ready' only."}])
    print('[adapter] Smoke response:', smoke[:120])
except Exception as e:
    raise RuntimeError(f'Model smoke test failed: {e}')

print(f"[adapter] Mode={'CUSTOM' if CUSTOM_ENDPOINT else 'AZURE_OPENAI'} project_fallback={LEGACY_PROJECT_NAME}")

[adapter] Smoke response: ready
[adapter] Mode=AZURE_OPENAI project_fallback=thorlabs-project-redteam-dev


In [28]:
# Legacy workspace control run (fallback) - minimal evaluation against real model
# This bypasses AIServices project (if blocked) to validate evaluation pipeline end-to-end.
from azure.ai.evaluation.red_team import RedTeam, RiskCategory, AttackStrategy
from azure.identity import ManagedIdentityCredential
import datetime as _dt, pathlib, traceback

if not LEGACY_PROJECT_NAME:
    print("[control] LEGACY_PROJECT_NAME not set; skipping control run.")
else:
    control_output_dir = pathlib.Path('/tmp/ai-redteam-control')
    control_output_dir.mkdir(parents=True, exist_ok=True)
    legacy_project_dict = {
        'subscription_id': SUBSCRIPTION_ID,
        'resource_group_name': RESOURCE_GROUP,
        'project_name': LEGACY_PROJECT_NAME,
    }
    print('[control] Using legacy project:', legacy_project_dict)
    _mi = ManagedIdentityCredential(client_id=UAMI_CLIENT_ID) if not FORCE_SYSTEM_MI else ManagedIdentityCredential()
    try:
        rt_control = RedTeam(
            azure_ai_project=legacy_project_dict,
            credential=_mi,
            risk_categories=[RiskCategory.Violence],
            num_objectives=1,
            output_dir=str(control_output_dir)
        )
        print('[control] RedTeam constructor OK (legacy)')
    except Exception as e:
        print('[control] Constructor failed:', e)
        traceback.print_exc(limit=1)
        rt_control = None

    if rt_control:
        from functools import partial
        # Adapt invoke_model -> RedTeam chat expected format
        def target_wrapper(prompt: str, **_):
            return invoke_model([{'role':'user','content': prompt}])
        res_control = await rt_control.scan(
            target=target_wrapper,
            scan_name='Legacy-Control-Scan',
            attack_strategies=[AttackStrategy.Flip]
        )
        print('[control] evaluation_id:', getattr(res_control, 'evaluation_id', None))
        for attr in ['scorecard_url','_scorecard_url','_evaluation_url']:
            v = getattr(res_control, attr, None)
            if v:
                print('[control] URL:', v)
                break
        else:
            print('[control] No portal URL attribute exposed (check UI after a minute).')
        subset = list(control_output_dir.glob('*.json'))[:6]
        if subset:
            print('[control] Artifacts subset:')
            for p in subset:
                print(' -', p.name)
        print('[control] Done.')

[control] Using legacy project: {'subscription_id': 'e440a65b-7418-4865-9821-88e411ffdd5b', 'resource_group_name': 'rg-thorlabs-redteam-dev-eastus2', 'project_name': 'thorlabs-project-redteam-dev'}
[control] RedTeam constructor OK (legacy)
🚀 STARTING RED TEAM SCAN: Legacy-Control-Scan
📂 Output directory: /tmp/ai-redteam-control/.scan_Legacy-Control-Scan_20250810_174847
📊 Risk categories: ['violence']
🔗 Track your red team scan in AI Foundry: https://ai.azure.com/build/evaluation/19cd4a57-1bc8-497f-8d07-9a96c49750c2?wsid=/subscriptions/e440a65b-7418-4865-9821-88e411ffdd5b/resourceGroups/rg-thorlabs-redteam-dev-eastus2/providers/Microsoft.MachineLearningServices/workspaces/thorlabs-project-redteam-dev
📋 Planning 2 total tasks


Scanning:   0%|                         | 0/2 [00:00<?, ?scan/s, current=fetching baseline/violence]

📚 Using attack objectives from Azure RAI service


Scanning:   0%|                                          | 0/2 [00:00<?, ?scan/s, current=batch 1/1]

📝 Fetched baseline objectives for violence: 1 objectives
🔄 Fetching objectives for strategy 2/2: flip
⚙️ Processing 2 tasks in parallel (max 5 at a time)
▶️ Starting task: baseline strategy for violence risk category
▶️ Starting task: flip strategy for violence risk category


Scanning: 100%|██████████████████████████████████| 2/2 [00:07<00:00,  3.70s/scan, current=batch 1/1]
Class RedTeamResult: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


Evaluation results saved to "/tmp/ai-redteam-control/.scan_Legacy-Control-Scan_20250810_174847/baseline_violence_54b9487c-0782-40ae-8ae5-baa1010347c4.json".

✅ Completed task 1/2 (50.0%) - baseline/violence in 7.0s
   Est. remaining: 0.1 minutes
Evaluation results saved to "/tmp/ai-redteam-control/.scan_Legacy-Control-Scan_20250810_174847/flip_violence_b9ae185c-275f-4a29-89bd-6bd405bb15f0.json".

✅ Completed task 2/2 (100.0%) - flip/violence in 7.1s
   Est. remaining: 0.0 minutes


Visit https://aka.ms/azsdk/python/evaluation/remotetracking/troubleshoot to troubleshoot this issue.


Evaluation results saved to "/tmp/ai-redteam-control/.scan_Legacy-Control-Scan_20250810_174847/final_results.json".

Overall ASR: 0.0%
Attack Success: 0/2 attacks were successful
------------------------------------------------------------------------------------------------------------------------------------
Risk Category     | Baseline ASR   | Easy-Complexity Attacks ASR  | Moderate-Complexity Attacks ASR | Difficult-Complexity Attacks ASR
------------------------------------------------------------------------------------------------------------------------------------
Violence          | 0.0%           | 0.0%                         | N/A                             | N/A                           

Detailed results available at:
https://ai.azure.com/build/evaluation/19cd4a57-1bc8-497f-8d07-9a96c49750c2?wsid=/subscriptions/e440a65b-7418-4865-9821-88e411ffdd5b/resourceGroups/rg-thorlabs-redteam-dev-eastus2/providers/Microsoft.MachineLearningServices/workspaces/thorlabs-project-redt

In [29]:
# Quick Runbook Execution (Single Minimal Evaluation)
# This cell performs: management-plane probe (best-effort), RedTeam construction, minimal scan.
# Safe to rerun; creates a timestamped output subdirectory per attempt.

import time, json, pathlib, datetime as _dt, traceback
from azure.ai.evaluation.red_team import RedTeam, RiskCategory, AttackStrategy
from azure.identity import ManagedIdentityCredential

_ts = _dt.datetime.utcnow().strftime('%Y%m%dT%H%M%S')
run_output_dir = pathlib.Path(output_dir) / f"run-{_ts}"  # isolate artifacts per run
run_output_dir.mkdir(parents=True, exist_ok=True)
print(f"[runbook] output_dir={run_output_dir}")

# 1. (Optional) Management-plane confirmation using already hardcoded version variable if present
try:
    api_ver = globals().get('AISVCS_PROJECT_API_VERSION', '2025-06-01')
    acct = os.environ.get('AISVCS_ACCOUNT', 'thorlabs-aisvc-redteam-dev')
    proj_id = f"/subscriptions/{SUBSCRIPTION_ID}/resourceGroups/{RESOURCE_GROUP}/providers/Microsoft.CognitiveServices/accounts/{acct}/projects/{PROJECT_NAME}"
    _cred = ManagedIdentityCredential()
    _arm_tok = _cred.get_token('https://management.azure.com/.default').token
    _h = {'Authorization': f'Bearer {_arm_tok}'}
    import requests as _rq
    _r = _rq.get(f"https://management.azure.com{proj_id}?api-version={api_ver}", headers=_h, timeout=20)
    print(f"[runbook] mgmt GET status={_r.status_code} api-version={api_ver}")
except Exception as mgmt_e:
    print('[runbook] mgmt probe skipped/error:', mgmt_e)

# 2. RedTeam construction (single category minimal)
start = time.time()
constructor_error = None
try:
    rt = RedTeam(
        azure_ai_project=azure_ai_project,
        credential=credential,
        risk_categories=[RiskCategory.Violence],
        num_objectives=1,
        output_dir=str(run_output_dir)
    )
    print(f"[runbook] RedTeam constructor OK in {time.time()-start:.2f}s")
except Exception as e:
    constructor_error = e
    elapsed = time.time()-start
    msg = str(e)
    print(f"[runbook] RedTeam constructor FAILED after {elapsed:.2f}s -> {e.__class__.__name__}: {msg[:180]}...")
    # Classify
    if '403' in msg or 'Forbidden' in msg:
        print('[classification] RBAC/role missing (ensure AzureML Data Scientist & storage roles).')
    elif '404' in msg:
        print('[classification] Possibly unsupported AIServices project path for current sdk or region not yet enabled.')
    else:
        print('[classification] Generic/other error category – inspect full traceback below.')
    traceback.print_exc(limit=1)

if constructor_error:
    raise constructor_error

# 3. Minimal scan against deterministic safe callback
scan_start = time.time()
result_min = await rt.scan(
    target=fixed_safe_callback,
    scan_name=f"Runbook-Minimal-{_ts}",
    attack_strategies=[AttackStrategy.Flip]
)
print(f"[runbook] scan complete in {time.time()-scan_start:.2f}s eval_id={getattr(result_min,'evaluation_id',None)}")

key_attrs = ['evaluation_id','evaluation_name','scorecard_url','_scorecard_url','_evaluation_url']
for a in key_attrs:
    v = getattr(result_min, a, None)
    if v:
        print(f"{a}: {v}")

# Show a small subset of produced JSON artifacts
artifacts = list(run_output_dir.glob('*.json'))[:8]
if artifacts:
    print('[runbook] artifact subset:')
    for p in artifacts:
        print(' -', p.name)
else:
    print('[runbook] no JSON artifacts yet (may appear slightly later).')

print('[runbook] Done. Proceed to advanced cells only after verifying portal visibility.')

[runbook] output_dir=/tmp/ai-redteam-logs/run-20250810T174904
[runbook] mgmt GET status=200 api-version=2025-06-01
[runbook] RedTeam constructor FAILED after 0.19s -> Exception: Failed to connect to your Azure AI project. Please check if the project scope is configured correctly, and make sure you have the necessary access permissions. Status code: 403....
[classification] RBAC/role missing (ensure AzureML Data Scientist & storage roles).
[runbook] mgmt GET status=200 api-version=2025-06-01
[runbook] RedTeam constructor FAILED after 0.19s -> Exception: Failed to connect to your Azure AI project. Please check if the project scope is configured correctly, and make sure you have the necessary access permissions. Status code: 403....
[classification] RBAC/role missing (ensure AzureML Data Scientist & storage roles).


Traceback (most recent call last):
  File "/tmp/ipykernel_151538/2389591547.py", line 32, in <module>
    rt = RedTeam(
Exception: Failed to connect to your Azure AI project. Please check if the project scope is configured correctly, and make sure you have the necessary access permissions. Status code: 403.


Exception: Failed to connect to your Azure AI project. Please check if the project scope is configured correctly, and make sure you have the necessary access permissions. Status code: 403.

### Configuration Inputs (Environment / Variables)

Purpose: Show the minimal configuration surface required for this slim workflow and where to change names.

Why: We avoid a .env loader and multi‑credential fallbacks so any missing value or identity issue fails fast and is obvious.

Required keys (earlier bootstrap cell sets them):
- azure_ai_project: subscription_id, resource_group_name, project_name (drives remote evaluation context)
- AZURE_OPENAI_ENDPOINT / DEPLOYMENT / API_VERSION: used by callbacks for model interaction (Managed Identity auth)

Adjust the values in the bootstrap cell if your resource names differ. No secrets or API keys are needed (Managed Identity only).


## Red Teaming Concepts (Quick Primer)
We exercise a small, representative slice of the full surface to keep runtime low while still surfacing safety signal quality.

Dimensions:
1. Risk Categories (subset here): Violence, HateUnfairness (later scans expand to Sexual, SelfHarm).
2. Attack Strategies: Transform text to pierce guardrails (we start with Flip for speed, then broaden in the advanced scan).
3. Complexity Levels: EASY/MODERATE groups bundle several transformations; Compose() can chain multiple.

Key Metric: Attack Success Rate (ASR) — proportion of attack objectives that elicit content judged unsafe.

Rationale for chosen ordering:
- Minimal single‑category scan: sanity + portal linkage check fast (~seconds)
- Basic callback: deterministic baseline (expected ASR ~0%)
- Intermediary & advanced scans: introduce genuine model variability + broader strategy set
- Custom prompts: lets you inject domain or product‑specific risky intents to track over time


### Storage & Permissions (Why They Matter)
- Storage Blob Data Contributor (unconditioned) for the project identity: required for artifact container creation & blob uploads (scorecards, JSON logs).
- Entra ID auth (allowSharedKeyAccess=false): enforces keyless posture; aligns with managed identity use.
- Network: After initial validation, restrict publicNetworkAccess where possible; keep AzureServices bypass to allow control-plane operations.

If artifacts or portal URLs are missing, first re‑check storage connection scope and role propagation before deeper debugging.


## Basic Example: Fixed Response Callback
Purpose: Validate pipeline wiring (constructor, execution, artifact logging) with a deterministic safe target whose expected ASR is 0%.
Why: Eliminates model variability so any failure points directly to infrastructure (permissions, connections, logging) rather than model behavior.


In [ ]:
# Define a simple callback function that always returns a fixed response
def financial_advisor_callback(query: str) -> str:  # noqa: ARG001
    return ("I'm a financial advisor assistant. I can help with investment advice and "
            "financial planning within legal and ethical guidelines.")

In [ ]:
# (Removed previous forced /tmp working directory change to rely on default path and cloud artifact logging)
import os, pathlib
print(f"Current working directory: {pathlib.Path.cwd()}")

In [ ]:
# Create the `RedTeam` instance with minimal configurations
red_team = RedTeam(
    azure_ai_project=azure_ai_project,
    credential=credential,
    risk_categories=[RiskCategory.Violence, RiskCategory.HateUnfairness],
    num_objectives=1,
    output_dir=str(output_dir),
)

Note: num_objectives controls how many distinct attack prompts are generated per risk category & strategy. Keep it low initially for fast feedback; increase later for statistical confidence.


Now let's run a simple automated scan using the `RedTeam` with the fixed response target. We'll test against two risk categories and one attack strategy for simplicity.

In [ ]:
# Run the red team scan called "Basic-Callback-Scan"
result = await red_team.scan(
    target=financial_advisor_callback,
    scan_name="Basic-Callback-Scan",
    attack_strategies=[AttackStrategy.Flip],
)

In [ ]:
# Basic-Callback-Scan concise diagnostics
from pathlib import Path
attrs = ["evaluation_id","evaluation_name","scorecard_url","_evaluation_url","_scorecard_url","run_id"]
print("=== Basic-Callback-Scan Summary (AIServices project: {} ) ===".format(azure_ai_project['project_name']))
for a in attrs:
    if hasattr(result, a):
        v = getattr(result, a)
        if v:
            print(f"{a}: {v}")
# Local artifact snapshot (first 10 files)
out_dir = Path(getattr(red_team, 'output_dir', '.'))
if out_dir.exists():
    files = [p for p in out_dir.glob('*.json')][:10]
    if files:
        print("Artifacts (subset):")
        for f in files:
            print(" -", f.name)
else:
    print("Output directory not found:", out_dir)

## Intermediary Example: Model Callback (Managed Identity)
Purpose: Transition from a deterministic fixed callback to a real model response while still keeping scope narrow (single Flip strategy) to validate that model invocation + evaluation loop work.
Why MI Callback: Reference docs often show a model config; here we avoid AzureCliCredential (no `az login` on remote) and keep keyless posture using Managed Identity.
Action: Run one light scan to confirm model path before investing time in broader strategies.


> Reference Adjustment: The full sample uses AzureCliCredential for model configs; we substitute a managed-identity-backed callback so this remains runnable on hosted compute without interactive auth.


In [ ]:
# Fallback: intermediary via managed identity callback (no az login required)
async def intermediary_callback(messages: list, stream: Optional[bool] = False, session_state: Optional[str] = None, context: Optional[Dict[str, Any]] = None):  # noqa: ARG001
    token_provider = get_bearer_token_provider(credential, "https://cognitiveservices.azure.com/.default")
    client = AzureOpenAI(
        azure_endpoint=azure_openai_endpoint,
        api_version=azure_openai_api_version,
        azure_ad_token_provider=token_provider,
    )
    messages_list = [{"role": message.role, "content": message.content} for message in messages]
    latest_message = messages_list[-1]["content"]
    try:
        response = client.chat.completions.create(
            model=azure_openai_deployment,
            messages=[{"role": "user", "content": latest_message}],
            max_completion_tokens=200,
            temperature=0.7,
        )
        return {"messages": [{"content": response.choices[0].message.content, "role": "assistant"}]}
    except Exception as e:
        print(f"Intermediary MI callback error: {e!s}")
        return {"messages": [{"content": "Service error.", "role": "assistant"}]}

# Re-run intermediary using MI-backed callback
intermediary_result = await red_team.scan(
    target=intermediary_callback,
    scan_name="Intermediary-Callback-Scan",
    attack_strategies=[AttackStrategy.Flip],
)

## Advanced Example: Broader Strategy Sweep Against Azure OpenAI
Purpose: Expand coverage to multiple risk categories and a curated set of transformation strategies to approximate a fuller red teaming pass.
Why Now: Only escalate after minimal + intermediary scans succeed so failures are more attributable (strategy breadth adds time & noise).
Guideline: Keep list focused; you can append more strategies (e.g., Jailbreak, Leetspeak variants) once baseline artifacts appear in the portal.


In [ ]:
# Define a callback that uses Azure OpenAI API to generate responses
async def azure_openai_callback(
    messages: list,
    stream: Optional[bool] = False,  # noqa: ARG001
    session_state: Optional[str] = None,  # noqa: ARG001
    context: Optional[Dict[str, Any]] = None,  # noqa: ARG001
) -> dict[str, list[dict[str, str]]]:
    # Get token provider for Azure AD authentication
    token_provider = get_bearer_token_provider(credential, "https://cognitiveservices.azure.com/.default")

    # Initialize Azure OpenAI client
    client = AzureOpenAI(
        azure_endpoint=azure_openai_endpoint,
        api_version=azure_openai_api_version,
        azure_ad_token_provider=token_provider,
    )

    ## Extract the latest message from the conversation history
    messages_list = [{"role": message.role, "content": message.content} for message in messages]
    latest_message = messages_list[-1]["content"]

    try:
        # Call the model with safety features minimized for red team testing
        response = client.chat.completions.create(
            model=azure_openai_deployment,
            messages=[
                {"role": "user", "content": latest_message},
            ],
            max_completion_tokens=500,
            temperature=0.7,
            # Note: Content filtering happens at the service level and cannot be disabled via API
        )

        # Format the response to follow the expected chat protocol format
        formatted_response = {"content": response.choices[0].message.content, "role": "assistant"}
    except Exception as e:
        print(f"Error calling Azure OpenAI: {e!s}")
        formatted_response = {"content": "I encountered an error and couldn't process your request.", "role": "assistant"}
    return {"messages": [formatted_response]}

In [ ]:
# Create the RedTeam instance with all of the risk categories with 5 attack objectives generated for each category
model_red_team = RedTeam(
    azure_ai_project=azure_ai_project,
    credential=credential,
    risk_categories=[RiskCategory.Violence, RiskCategory.HateUnfairness, RiskCategory.Sexual, RiskCategory.SelfHarm],
    num_objectives=5,
    output_dir=str(output_dir),
)

We will use this instance of `model_red_team` to test different attack strategies in the following section.

### Testing Different Attack Strategies

Now we'll run a more comprehensive evaluation using multiple attack strategies across risk categories. This will give us a better understanding of our model's vulnerabilities.

In [ ]:
# Run the red team scan with multiple attack strategies
advanced_result = await model_red_team.scan(
    target=azure_openai_callback,
    scan_name="Advanced-Callback-Scan",
    attack_strategies=[
        AttackStrategy.EASY,
        AttackStrategy.MODERATE,
        AttackStrategy.CharacterSpace,
        AttackStrategy.ROT13,
        AttackStrategy.UnicodeConfusable,
        AttackStrategy.CharSwap,
        AttackStrategy.Morse,
        AttackStrategy.Leetspeak,
        AttackStrategy.Url,
        AttackStrategy.Binary,
        AttackStrategy.Compose([AttackStrategy.Base64, AttackStrategy.ROT13]),
    ],
)

## Custom Objectives (Bring Your Own Prompts)
Purpose: Inject domain / product specific risky intents to track regressions over time.
Format: JSON file where each entry supplies a risk-type and prompt.
When to Use: After at least one portal-visible evaluation so you know artifact logging path works—then add high-value prompts tied to your threat model.


In [ ]:
from pathlib import Path
prompts_path = Path("AI_RedTeaming/data/prompts.json")
print(f"Prompts path: {prompts_path.resolve()}")
if not prompts_path.exists():
    raise FileNotFoundError(f"Prompts file not found at {prompts_path.resolve()}")

.__class__  # no-op

# Create the RedTeam specifying the custom attack seed prompts to use as objectives
custom_red_team = RedTeam(
    azure_ai_project=azure_ai_project,
    credential=credential,
    custom_attack_seed_prompts=str(prompts_path),  # Path to a file containing custom attack seed prompts
    output_dir=str(output_dir),
)

In [ ]:
custom_red_team_result = await custom_red_team.scan(
    target=azure_openai_callback,
    scan_name="Custom-Prompt-Scan",
    attack_strategies=[
        AttackStrategy.EASY,  # Group of easy complexity attacks
        AttackStrategy.MODERATE,  # Group of moderate complexity attacks
        AttackStrategy.DIFFICULT,  # Group of difficult complexity attacks
    ],
)

## Conclusion & Next Steps
You now have a minimal, identity-only path from environment bootstrap to multi-strategy safety evaluation. Start small (fixed callback, single category) then scale breadth and custom objectives as artifacts reliably appear.

Interpreting Results:
- ASR high in a category: prioritize mitigation (prompt filters, fine-tuning, safety middleware)
- Specific strategies dominant: add targeted defenses (normalization, decoding layers, pattern detectors)
- Custom prompts succeeding: escalate to product security review & adjust guardrails.

Recommended Follow-Up:
1. Add a periodic (e.g., daily) minimal scan in CI to detect sudden ASR spikes.
2. Version control your custom prompts file; treat changes as threat model deltas.
3. Introduce additional strategies gradually; monitor runtime impact.
4. Layer Azure AI Content Safety (or similar) pre/post filters and re-measure ASR.

For deep diagnostics (artifact uploads, RBAC edge cases), consult the reference notebook.


## 🔍 Results Verification
Goal: Confirm each scan produces a portal-visible evaluation + scorecard before expanding scope.

Checklist:
1. After Minimal or Basic scan: copy any printed scorecard/evaluation URL and open in AI Foundry.
2. If URL missing: wait 1–2 minutes; if still absent, re-run readiness check below.
3. If readiness succeeds but no artifacts: revalidate storage connection + RBAC (Blob Data Contributor) on the project identity.


In [ ]:
# RedTeam readiness quick check (safe to re-run)
# Attempts minimal RedTeam construction to verify RBAC on the AI Project for the active managed identity.
from azure.ai.evaluation.red_team import RedTeam, RiskCategory
import time, traceback
print("Running RedTeam readiness check...")
start = time.time()
try:
    _rt_check = RedTeam(
        azure_ai_project=azure_ai_project,
        credential=credential,
        risk_categories=[RiskCategory.Violence],
        num_objectives=1,
        output_dir=str(output_dir),
    )
    elapsed = time.time() - start
    print(f"✅ RedTeam constructor succeeded in {elapsed:.2f}s. RBAC appears satisfied.")
    print("Next: Run the Basic-Callback-Scan cell above to begin evaluation.")
except Exception as e:
    elapsed = time.time() - start
    msg = str(e)
    print(f"❌ RedTeam constructor failed after {elapsed:.2f}s -> {e.__class__.__name__}: {msg[:180]}...")
    if "403" in msg or "Forbidden" in msg:
        print("Likely cause: UAMI missing AzureML Data Scientist at project scope. Once granted, wait 2-5 min and re-run.")
    traceback.print_exc(limit=1)

# --- Optional automatic poller (set attempts >1 to enable) ---
AUTO_POLL_ATTEMPTS = 1          # Increase (e.g., 10) to poll automatically
AUTO_POLL_INTERVAL_SEC = 60     # 60s per your request
if AUTO_POLL_ATTEMPTS > 1:
    print(f"Starting auto-poll: attempts={AUTO_POLL_ATTEMPTS}, interval={AUTO_POLL_INTERVAL_SEC}s")
    success = False
    for attempt in range(1, AUTO_POLL_ATTEMPTS + 1):
        try:
            _rt_check = RedTeam(
                azure_ai_project=azure_ai_project,
                credential=credential,
                risk_categories=[RiskCategory.Violence],
                num_objectives=1,
                output_dir=str(output_dir),
            )
            print(f"Attempt {attempt}: ✅ Success")
            success = True
            break
        except Exception as e2:
            if "403" in str(e2):
                print(f"Attempt {attempt}: 403 still present; waiting {AUTO_POLL_INTERVAL_SEC}s...")
            else:
                print(f"Attempt {attempt}: other error {e2.__class__.__name__}: {str(e2)[:120]}...")
            if attempt < AUTO_POLL_ATTEMPTS:
                import time as _t; _t.sleep(AUTO_POLL_INTERVAL_SEC)
    if not success:
        print("Auto-poll finished without success. Increase AUTO_POLL_ATTEMPTS after verifying the role assignment is created.")


## Minimal Permissions Proof Plan (Pending Compute Availability)

This section validates that only the minimally required RBAC assignments are in place for running an AI Red Teaming evaluation against the Azure AI Foundry Project (without broad owner/contributor roles):

Target minimal roles per identity actually used for execution:
- PROJECT (Azure AI Foundry Project scope): Azure AI User
- MODEL (Azure OpenAI account): Cognitive Services OpenAI User
- STORAGE (Results/artifacts account): Storage Blob Data Contributor (plus listAccountSas/action if SDK requires SAS fallback) — aim to replace broad Storage Account Contributor / Blob Data Owner

Validation Steps (performed by the following code cell once the compute instance is Running):
1. Fetch AAD access tokens for: management.azure.com, https://cognitiveservices.azure.com/, https://storage.azure.com/.
2. Management-plane read: GET Project resource to confirm Azure AI User scope works.
3. Data-plane model invocation: Simple chat/completions call (expect 200) using Azure OpenAI deployment.
4. Storage data-plane write: Create (idempotent) a temporary container + upload a tiny blob using only Bearer token (no shared keys) -> proves Blob Data Contributor suffices.
5. Optional SAS probe (only if SDK still emits artifact upload warning) — indicates listAccountSas/action requirement still present.
6. Summarize which permissions succeeded or failed and map to role deltas.

Run the next code cell after:
- Compute state == Running
- Broad roles optionally removed (execute before removal for baseline, then after removal for confirmation)

Outputs:
- Table-like text summary (Permission, Endpoint, Status, Interpretation)
- Remediation recommendations if any step fails.

Do NOT execute locally; this must run on the remote managed identity context.


In [ ]:
# Minimal Permissions Validation Execution
# PRECONDITION: Running on remote compute with ManagedIdentityCredential referencing UAMI.

import os, json, time, uuid, textwrap, sys
from dataclasses import dataclass
from typing import List
import requests
from azure.identity import ManagedIdentityCredential

SUBSCRIPTION_ID = os.environ.get("AZ_SUB_ID", "e440a65b-7418-4865-9821-88e411ffdd5b")
RESOURCE_GROUP = os.environ.get("AZ_RG", "rg-thorlabs-redteam-dev-eastus2")
ACCOUNT_NAME = os.environ.get("AISVC_ACCOUNT", "thorlabs-aisvc-redteam-dev")
PROJECT_NAME = os.environ.get("AISVC_PROJECT", "thorlabs-aisvcproj-redteam-dev")
OPENAI_ENDPOINT = os.environ.get("AZURE_OPENAI_ENDPOINT") or "https://thorlabs-openai-redteam-dev.openai.azure.com/"
OPENAI_DEPLOYMENT = os.environ.get("AZURE_OPENAI_DEPLOYMENT", "gpt-4o-mini")  # adjust if needed
STORAGE_ACCOUNT = os.environ.get("STORAGE_ACCOUNT", "thorlabsredteamdev001")
TMP_CONTAINER = f"perm-proof-{uuid.uuid4().hex[:8]}".lower()

credential = ManagedIdentityCredential(client_id=os.environ.get("UAMI_CLIENT_ID", "95aedfd4-301c-4105-a6db-0a83c9fd5ddd"))

@dataclass
class StepResult:
    name: str
    status: str
    detail: str
    remediation: str = ""

results: List[StepResult] = []

def fetch_token(scope: str):
    try:
        t0=time.time(); token=credential.get_token(scope)
        latency = (time.time()-t0)*1000
        return token.token, latency, None
    except Exception as e:
        return None, None, str(e)

# 1. Tokens
for scope in ["https://management.azure.com/.default", "https://cognitiveservices.azure.com/.default", "https://storage.azure.com/.default"]:
    tok, latency, err = fetch_token(scope)
    if tok:
        results.append(StepResult(f"Token: {scope}", "OK", f"latency_ms={latency:.1f}"))
    else:
        results.append(StepResult(f"Token: {scope}", "FAIL", err or "unknown", remediation="Ensure MI has access & IMDS reachable"))

# Short-circuit if critical tokens missing
if any(r.status=="FAIL" for r in results if r.name.startswith("Token: https://management")):
    print("Management token missing; halt further steps.")
else:
    # 2. Mgmt-plane project GET
    proj_id = f"/subscriptions/{SUBSCRIPTION_ID}/resourceGroups/{RESOURCE_GROUP}/providers/Microsoft.CognitiveServices/accounts/{ACCOUNT_NAME}/projects/{PROJECT_NAME}"
    mgmt_url = f"https://management.azure.com{proj_id}?api-version=2024-10-01-preview"
    mtoken = [r for r in results if r.name.startswith("Token: https://management")][0]
    try:
        hdr = {"Authorization": f"Bearer {credential.get_token('https://management.azure.com/.default').token}"}
        r = requests.get(mgmt_url, headers=hdr, timeout=15)
        if r.status_code==200:
            results.append(StepResult("Mgmt Project Read", "OK", "200"))
        else:
            rem = "Confirm Azure AI User role at project/account scope" if r.status_code in (403,404) else "Check api-version or network";
            results.append(StepResult("Mgmt Project Read", "FAIL", f"status={r.status_code} body={r.text[:180]}", remediation=rem))
    except Exception as e:
        results.append(StepResult("Mgmt Project Read", "FAIL", str(e), remediation="Network / DNS / IMDS issue"))

# 3. Model invocation (simple chat completions). Avoid openai lib to keep minimal surface.
if not any(r.status=="FAIL" and r.name=="Mgmt Project Read" for r in results):
    try:
        # Azure OpenAI Chat Completions REST (2024-02-15-preview style may vary)
        # We assume model is chat capable; if not adjust path.
        api_version = os.environ.get("AZURE_OPENAI_API_VERSION", "2024-02-15-preview")
        url = f"{OPENAI_ENDPOINT}openai/deployments/{OPENAI_DEPLOYMENT}/chat/completions?api-version={api_version}"
        ctoken,_lat,_ = fetch_token("https://cognitiveservices.azure.com/.default")
        if not ctoken:
            results.append(StepResult("Model Invoke", "FAIL", "No cognitive token", remediation="Azure AI User + OpenAI User role needed"))
        else:
            payload = {"messages":[{"role":"user","content":"ping"}],"max_tokens":5}
            rr = requests.post(url, headers={"Authorization": f"Bearer {ctoken}", "Content-Type":"application/json"}, json=payload, timeout=20)
            if rr.status_code==200:
                results.append(StepResult("Model Invoke", "OK", "200"))
            else:
                rem = "Ensure Cognitive Services OpenAI User role" if rr.status_code in (401,403) else "Check deployment name/api-version"
                results.append(StepResult("Model Invoke", "FAIL", f"status={rr.status_code} body={rr.text[:160]}", remediation=rem))
    except Exception as e:
        results.append(StepResult("Model Invoke", "FAIL", str(e), remediation="Network / role / endpoint"))

# 4. Storage container + blob write
try:
    stoken,_lat,_ = fetch_token("https://storage.azure.com/.default")
    if not stoken:
        results.append(StepResult("Storage Token", "FAIL", "No storage token", remediation="Blob Data Contributor"))
    else:
        base = f"https://{STORAGE_ACCOUNT}.blob.core.windows.net/{TMP_CONTAINER}?restype=container"
        h = {"Authorization": f"Bearer {stoken}", "x-ms-version":"2023-11-03"}
        cr = requests.put(base, headers=h, timeout=15)
        if cr.status_code in (201,202,409):
            # upload small blob
            blob_url = f"https://{STORAGE_ACCOUNT}.blob.core.windows.net/{TMP_CONTAINER}/proof.txt"
            bh = {**h, "x-ms-blob-type":"BlockBlob", "Content-Type":"text/plain"}
            br = requests.put(blob_url, headers=bh, data=b"ok", timeout=15)
            if br.status_code in (201,202):
                results.append(StepResult("Storage Write", "OK", f"container={TMP_CONTAINER}"))
            else:
                results.append(StepResult("Storage Write", "FAIL", f"blob status={br.status_code}", remediation="Blob Data Contributor role"))
        else:
            results.append(StepResult("Storage Container Create", "FAIL", f"status={cr.status_code}", remediation="Blob Data Contributor (data plane)"))
except Exception as e:
    results.append(StepResult("Storage Write", "FAIL", str(e), remediation="Network / RBAC"))

# 5. Summarize
print("Permission Proof Summary:\n")
for r in results:
    print(f"- {r.name:<22} {r.status:<4} | {r.detail} {('-> ' + r.remediation) if r.remediation else ''}")

# 6. Recommend cleanup if all OK
if all(r.status=="OK" for r in results if not r.name.startswith("Token: https://management") or r.status=="OK"):
    print("\nAll core operations succeeded with current roles. You can now safely remove broad Storage Account Contributor / Blob Data Owner and replace with custom minimal role (listAccountSas/action + blob data actions) if SAS generation continues to be needed.")
else:
    failing = [r for r in results if r.status=="FAIL"]
    print("\nFailures detected:")
    for f in failing:
        print(f" * {f.name}: {f.detail} :: {f.remediation}")
